In [13]:
import sys
!{sys.executable} -m pip install langchain-community langchain-ollama langchain-core faiss-cpu

  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_ollama-1.0.1-py3-none-any.whl.metadata (2.5 kB)
  Using cached langchain_core-1.2.7-py3-none-any.whl.metadata (3.7 kB)
  Using cached langchain_classic-1.0.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached tenacity-9.1.2-py3-none-any.whl.metadata (1.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached pydantic_settings-2.12.0-py3-none-any.whl.metadata (3.4 kB)
  Using cached httpx_sse-0.4.3-py3-none-any.whl.metadata (9.7 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached pydantic-2.12.5-py3-none-any.whl.metadata (90 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached marshmallow-


[notice] A new release of pip is available: 25.2 -> 26.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings # Import spécifique à Ollama
from langchain_community.vectorstores import FAISS
import os
from langchain_community.document_loaders import TextLoader
from pathlib import Path
print("Modules importés avec succès.")

c:\Users\mohaa\AppData\Local\Programs\Python\Python314\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


Modules importés avec succès.


In [15]:
# Définition des chemins et des paramètres du modèle
knowledge_base = Path(r"C:\Users\mohaa\Desktop\IA\AI-Tutor\cnn_knowledge_base_eng")
nom_index_faiss = "faiss_index_pfe"
modele_ollama = "mxbai-embed-large"

# Récupération de la liste des fichiers .txt
files = list(knowledge_base.glob("**/*.txt"))
print(f"{len(files)} fichiers trouvés dans {knowledge_base}...")

199 fichiers trouvés dans C:\Users\mohaa\Desktop\IA\AI-Tutor\cnn_knowledge_base_eng...


In [16]:
tous_les_documents = []

# Boucle de chargement individuel
for file_path in files:
    try:
        loader = TextLoader(str(file_path), encoding="utf-8") 
        documents = loader.load()
        
        # Ajout à la liste globale
        tous_les_documents.extend(documents)
        print(f"Chargé : {file_path.name}")
        
    except Exception as e:
        print(f"Erreur sur {file_path.name} : {e}")

print(f"\nTotal : {len(tous_les_documents)} documents chargés.")

Chargé : 01_introduction_cnn.txt
Chargé : 02_convolution_layers.txt
Chargé : 03_pooling_layers.txt
Chargé : 04_activation_functions.txt
Chargé : 05_resnet_architecture.txt
Chargé : 06_batch_normalization.txt
Chargé : 07_regularization_dropout.txt
Chargé : 08_transfer_learning.txt
Chargé : 09_data_augmentation.txt
Chargé : 10_optimization_hyperparameters.txt
Chargé : 16_pytorch_implementation.txt
Chargé : 17_onnx_tensorrt_deployment.txt
Chargé : 18_debugging_visualization.txt
Chargé : 19_modern_architectures.txt
Chargé : 20_best_practices.txt
Chargé : 21_history_evolution.txt
Chargé : 22_theoretical_concepts.txt
Chargé : 23_real_applications.txt
Chargé : 24_architecture_comparisons.txt
Chargé : 25_challenges_limitations.txt
Chargé : 26_general_faq.txt
Chargé : 27_trends_future.txt
Chargé : 28_learning_resources.txt
Chargé : 29_common_mistakes.txt
Chargé : 30_glossary.txt
Chargé : docs_pytorch_org_tutorials_recipes_indexhtml.txt
Chargé : pytorch_org_announcements.txt
Chargé : pytorch_org

In [17]:
if tous_les_documents:
    # Configuration du splitter
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=900, 
        chunk_overlap=90
    )
    
    # Création des segments
    docs = text_splitter.split_documents(tous_les_documents)
    print(f"Génération de {len(docs)} chunks terminée.")
else:
    print("Aucun document à traiter.")

Génération de 3629 chunks terminée.


In [ ]:
# Initialisation du modèle d'embedding 
embeddings = OllamaEmbeddings(model=modele_ollama)

print("Vectorisation et indexation en cours (patience)...")
# Création de la base de données à partir des documents
vector_db = FAISS.from_documents(docs, embeddings)

# Sauvegarde locale de l'index pour réutilisation ultérieure
vector_db.save_local(nom_index_faiss)
print(f"Index sauvegardé avec succès dans le dossier '{nom_index_faiss}'")